In [ ]:
Data Preparation section

SyntaxError: invalid syntax (<ipython-input-1-2595904377>, line 1)

In [ ]:
# 1. Install and imports
!pip install --upgrade pip
!pip install fastai pandas numpy scikit-learn --quiet

from fastai.tabular.all import *
import pandas as pd
from google.colab import drive; drive.mount('/content/drive')

# 2. File paths for all FD001–FD004
base_dir = '/content/drive/MyDrive/rul_project'
fds = [1, 2, 3, 4]
train_files = [f'{base_dir}/P_Train{fd}.csv' for fd in fds]
test_files  = [f'{base_dir}/P_Test{fd}.csv'  for fd in fds]

# 3. Load and store dataframes for all FDs in dictionaries
train_dfs = {}
test_dfs = {}
tabular_objs = {}
dls_objs = {}

for i, fd in enumerate(fds, 1):
    # Load, clean columns
    tdf = pd.read_csv(train_files[i-1])
    tdf.columns = tdf.columns.str.strip().str.lower()
    train_dfs[fd] = tdf

    tsdf = pd.read_csv(test_files[i-1])
    tsdf.columns = tsdf.columns.str.strip().str.lower()
    test_dfs[fd] = tsdf

    print(f"\n==== FD00{fd} ====")
    print("Train columns:", train_dfs[fd].columns.tolist())
    print(train_dfs[fd].head())
    print("Test columns:", test_dfs[fd].columns.tolist())
    print(test_dfs[fd].head())

    # Set up TabularPandas for each FD (if you want to do training right away)
    cat_names = []
    cont_names = [c for c in train_dfs[fd].columns if c != 'rul']
    procs = [FillMissing, Normalize]
    splits = RandomSplitter(valid_pct=0.2, seed=42)(train_dfs[fd])
    tabular_objs[fd] = TabularPandas(
        train_dfs[fd],
        procs,
        cat_names,
        cont_names,
        y_names='rul',
        splits=splits
    )
    dls_objs[fd] = tabular_objs[fd].dataloaders(bs=32)  # or adjust bs as needed

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 131.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 140.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 M



```
# This is formatted as code
```

Training and tuning - exploration

In [ ]:
from fastai.callback.all import CSVLogger, SaveModelCallback
import pandas as pd

# 1. Set up grid search parameters
layer_grid = [
    [200, 100],
    [400, 200],
    [400, 200, 100]
]
bs_grid = [8, 16, 32]
epoch_grid = [10, 20, 30]
procs = [FillMissing, Normalize]
cat_names = []

all_results = []

# 2. Loop through each FD scenario
for fd in [1, 2, 3, 4]:
    print(f"\n=== Grid Search for FD00{fd} ===")
    train_df = train_dfs[fd]
    test_df = test_dfs[fd]
    cont_names = [c for c in train_df.columns if c != 'rul']
    fd_results = []
    best_mae = float('inf')
    best_config = {}
    # Try several splits to ensure valid split isn't too small for chosen bs
    for bs in bs_grid:
        splits = RandomSplitter(valid_pct=0.2, seed=42)(train_df)
        valid_size = len(splits[1])
        print(f"  FD00{fd} | bs={bs} | valid set size: {valid_size}")
        if valid_size < bs:
            print(f"    SKIP: bs={bs} too big for valid set ({valid_size} samples)")
            continue
        tab = TabularPandas(
            train_df,
            procs,
            cat_names,
            cont_names,
            y_names='rul',
            splits=splits
        )
        dls = tab.dataloaders(bs=bs)
        for layers in layer_grid:
            for n_epochs in epoch_grid:
                try:
                    learn = tabular_learner(
                        dls,
                        layers=layers,
                        loss_func=MSELossFlat(),
                        metrics=mae,
                        cbs=[
                            CSVLogger(),
                            SaveModelCallback(monitor='mae', comp=np.less, fname=f'best_fd00{fd}', reset_on_fit=True)
                        ]
                    )
                    lr_valley = learn.lr_find().valley
                    for lr in [lr_valley, lr_valley * 0.3, lr_valley * 3]:
                        print(f"  FD00{fd} | bs={bs} | layers={layers} | epochs={n_epochs} | lr={lr:.6f}")
                        learn.fit_one_cycle(n_epochs, lr_max=lr)
                        val_metrics = learn.validate()
                        if len(val_metrics) > 1:
                            val_mae = float(val_metrics[1])
                        else:
                            val_mae = float('inf')
                        fd_results.append({
                            "fd": fd,
                            "bs": bs,
                            "layers": str(layers),
                            "epochs": n_epochs,
                            "lr": float(lr),
                            "val_mae": val_mae
                        })
                        # Save best config details (model is already saved by SaveModelCallback)
                        if val_mae < best_mae:
                            best_mae = val_mae
                            best_config = {
                                "fd": fd, "bs": bs, "layers": layers, "epochs": n_epochs, "lr": float(lr)
                            }
                        print(f"    -> Validation MAE: {val_mae:.2f}")
                except Exception as e:
                    print(f"  SKIP config due to error: {e}")
                    continue
    all_results.extend(fd_results)
    print(f"Best config for FD00{fd}: {best_config}, Best Validation MAE: {best_mae}")
    print(f"Best model for FD00{fd} saved as 'best_fd00{fd}.pth'")

# 3. Save leaderboard
results_df = pd.DataFrame(all_results).sort_values(["fd", "val_mae"])
results_df.to_csv(f"{base_dir}/all_fd_grid_search_leaderboard.csv", index=False)
print("\n=== Overall Grid Search Leaderboard (Top 10) ===")
print(results_df.head(10))

In [ ]:
2nd iteration on fine-tuning

In [ ]:
from fastai.tabular.all import *
from fastai.callback.all import CSVLogger, SaveModelCallback, EarlyStoppingCallback
import pandas as pd
import numpy as np

# === Hyperparameter grids reflecting your Optuna space ===

bs_grid = [16, 32, 64]
lr_grid = [1e-4, 3e-4, 1e-3, 3e-3, 1e-2]  # log scale within 1e-4 to 1e-2
wd_grid = [1e-5, 3e-5, 1e-4, 3e-4, 1e-3]  # log scale within 1e-5 to 1e-3
drop_grid = [0.05, 0.10, 0.15, 0.20]
layer_grid = [
    [400, 200, 100],
    [200, 100],
    [800, 400, 200],
    [256, 128],
    [512, 256, 128]
]
n_epochs = 30  # Consistent with your objective

procs = [FillMissing, Normalize]
cat_names = []
fd = 2  # FD002

# --- LOAD YOUR DATA (assumes train_dfs and test_dfs are already loaded) ---
train_df = train_dfs[fd]
test_df = test_dfs[fd]
cont_names = [c for c in train_df.columns if c != 'rul']

all_results = []
best_mae = float('inf')
best_config = {}

# ---- GRID SEARCH LOOP ----
print(f"\n=== Full Grid/Random Search for FD002 (mirrors Optuna space) ===")
for bs in bs_grid:
    splits = RandomSplitter(valid_pct=0.2, seed=42)(train_df)
    valid_size = len(splits[1])
    print(f"  FD002 | bs={bs} | valid set size: {valid_size}")
    if valid_size < bs:
        print(f"    SKIP: bs={bs} too big for valid set ({valid_size} samples)")
        continue
    tab = TabularPandas(
        train_df, procs, cat_names, cont_names, y_names='rul', splits=splits
    )
    dls = tab.dataloaders(bs=bs)
    for layers in layer_grid:
        for wd in wd_grid:
            for drop in drop_grid:
                ps = [drop] * len(layers)  # one dropout value per layer
                config = tabular_config(ps=ps)
                for lr in lr_grid:
                    try:
                        learn = tabular_learner(
                            dls,
                            layers=layers,
                            config=config,
                            wd=wd,
                            loss_func=MSELossFlat(),
                            metrics=mae,
                            cbs=[
                                CSVLogger(),
                                SaveModelCallback(monitor='mae', comp=np.less, fname=f'best_fd002', reset_on_fit=True),
                                EarlyStoppingCallback(monitor='mae', patience=25)
                            ]
                        )
                        print(f"bs={bs} | layers={layers} | wd={wd} | drop={drop} | lr={lr}")
                        learn.fit_one_cycle(n_epochs, lr_max=lr)
                        val_metrics = learn.validate()
                        val_mae = float(val_metrics[1]) if len(val_metrics) > 1 else float('inf')
                        config_result = {
                            "bs": bs,
                            "layers": str(layers),
                            "epochs": n_epochs,
                            "lr": float(lr),
                            "wd": float(wd),
                            "drop": float(drop),
                            "val_mae": val_mae
                        }
                        all_results.append(config_result)
                        if val_mae < best_mae:
                            best_mae = val_mae
                            best_config = config_result.copy()
                        print(f"    -> Validation MAE: {val_mae:.2f}")
                    except Exception as e:
                        print(f"  SKIP config due to error: {e}")
                        continue

print(f"\nBest config for FD002: {best_config}, Best Validation MAE: {best_mae}")
print(f"Best model for FD002 saved as 'best_fd002.pth'")

# --- SAVE RESULTS TO CSV ---
results_df = pd.DataFrame(all_results).sort_values("val_mae")
results_df.to_csv(f"{base_dir}/fd002_grid_search_leaderboard.csv", index=False)
print("\n=== FD002 Grid Search Leaderboard (Top 10) ===")
print(results_df.head(10))



=== Full Grid/Random Search for FD002 (mirrors Optuna space) ===
  FD002 | bs=16 | valid set size: 10751
bs=16 | layers=[400, 200, 100] | wd=1e-05 | drop=0.05 | lr=0.0001


epoch,train_loss,valid_loss,mae,time
0,16118.655273,16024.066406,108.074722,00:32
1,15516.037109,15552.613281,107.556610,00:31
2,13665.036133,14375.030273,105.085007,00:32
3,12315.727539,12221.643555,99.537987,00:33
4,9634.542969,9866.000000,91.000870,00:31
5,6468.525879,6420.466309,69.446327,00:33
6,2783.645020,2245.289795,33.769421,00:31
7,1936.242676,1526.511108,28.548546,00:32
8,1816.421021,1520.852417,29.441374,00:32
9,1628.236328,1575.340698,30.388292,00:31


Better model found at epoch 0 with mae value: 108.07472229003906.
Better model found at epoch 1 with mae value: 107.55661010742188.
Better model found at epoch 2 with mae value: 105.08500671386719.
Better model found at epoch 3 with mae value: 99.5379867553711.
Better model found at epoch 4 with mae value: 91.00086975097656.
Better model found at epoch 5 with mae value: 69.44632720947266.
Better model found at epoch 6 with mae value: 33.7694206237793.
Better model found at epoch 7 with mae value: 28.548545837402344.
No improvement since epoch 0: early stopping


  SKIP config due to error: Exception occured in `SaveModelCallback` when calling event `after_epoch`:
	list index out of range
bs=16 | layers=[400, 200, 100] | wd=1e-05 | drop=0.05 | lr=0.0003


epoch,train_loss,valid_loss,mae,time
0,14873.845703,15498.495117,107.225098,00:31
1,14591.931641,14484.379883,106.947296,00:31
2,10932.015625,9820.560547,90.259781,00:32
3,4541.319824,4330.466797,52.655315,00:33
4,1833.479370,1554.389282,29.120491,00:33
5,1786.446777,1556.580933,30.779922,00:32
6,1752.165283,1581.402344,30.189314,00:32
7,1727.628540,1539.941528,29.064671,00:32
8,1785.948242,1645.404907,31.205564,00:32
9,1611.713867,1522.173584,29.425629,00:31


...

epoch,train_loss,valid_loss,mae,time
0,10330.816406,9838.649414,90.276871,00:37
1,1833.616821,1714.047241,30.185354,00:36
2,1836.956299,1577.623657,30.270832,00:36
3,1904.379639,1626.921631,30.001364,00:36
4,1900.232666,1695.515991,31.625944,00:34
5,1651.429199,1503.461426,29.348110,00:35
6,1774.244385,1596.988770,30.422838,00:35
7,1907.727295,1558.895386,28.936630,00:36
8,1753.076172,1746.628540,32.477539,00:36
9,1849.636353,1853.799438,33.765247,00:35


Better model found at epoch 0 with mae value: 90.27687072753906.
Better model found at epoch 1 with mae value: 30.185354232788086.
Better model found at epoch 3 with mae value: 30.00136375427246.
Better model found at epoch 5 with mae value: 29.34811019897461.
Better model found at epoch 7 with mae value: 28.936630249023438.
Better model found at epoch 10 with mae value: 28.554363250732422.
Better model found at epoch 25 with mae value: 28.374263763427734.
No improvement since epoch 0: early stopping


  SKIP config due to error: Exception occured in `SaveModelCallback` when calling event `after_epoch`:
	list index out of range
bs=16 | layers=[400, 200, 100] | wd=0.0001 | drop=0.05 | lr=0.01


epoch,train_loss,valid_loss,mae,time
0,1874.775391,1663.441040,30.632439,00:36
1,1774.229370,1720.861328,32.398781,00:36
2,2075.938721,1615.874268,29.295065,00:34
3,2048.227051,1651.277832,29.575272,00:36
4,1801.619873,1724.485962,30.729235,00:35
5,1956.650635,1575.035034,29.927530,00:36
6,1780.583252,1693.896362,30.468567,00:35
7,2024.262329,399571.937500,48.784229,00:34


Better model found at epoch 0 with mae value: 30.63243865966797.
Better model found at epoch 2 with mae value: 29.29506492614746.


3rd iteration on fine-tuning

In [ ]:
from fastai.tabular.all import *
from fastai.callback.all import CSVLogger, SaveModelCallback, EarlyStoppingCallback
import pandas as pd
import numpy as np

# === Hyperparameter grids reflecting your Optuna space ===

bs_grid = [32, 64]
lr_grid = [0.0003, 0.001, 0.003]  # log scale within 1e-4 to 1e-2
wd_grid = [1e-5, 3e-5, 1e-4, 3e-4]  # log scale within 1e-5 to 1e-3
drop_grid = [0.08, 0.12, 0.16]
layer_grid = [[
   800, 400, 200], [256, 128], [512, 256, 128]]

n_epochs = 30  # Consistent with your objective

procs = [FillMissing, Normalize]
cat_names = []
fd = 2  # FD002

# --- LOAD YOUR DATA (assumes train_dfs and test_dfs are already loaded) ---
train_df = train_dfs[fd]
test_df = test_dfs[fd]
cont_names = [c for c in train_df.columns if c != 'rul']

all_results = []
best_mae = float('inf')
best_config = {}

# ---- GRID SEARCH LOOP ----
print(f"\n=== Full Grid/Random Search for FD002 (mirrors Optuna space) ===")
for bs in bs_grid:
    splits = RandomSplitter(valid_pct=0.2, seed=42)(train_df)
    valid_size = len(splits[1])
    print(f"  FD002 | bs={bs} | valid set size: {valid_size}")
    if valid_size < bs:
        print(f"    SKIP: bs={bs} too big for valid set ({valid_size} samples)")
        continue
    tab = TabularPandas(
        train_df, procs, cat_names, cont_names, y_names='rul', splits=splits
    )
    dls = tab.dataloaders(bs=bs)
    for layers in layer_grid:
        for wd in wd_grid:
            for drop in drop_grid:
                ps = [drop] * len(layers)  # one dropout value per layer
                config = tabular_config(ps=ps)
                for lr in lr_grid:
                    try:
                        learn = tabular_learner(
                            dls,
                            layers=layers,
                            config=config,
                            wd=wd,
                            loss_func=MSELossFlat(),
                            metrics=mae,
                            cbs=[
                                CSVLogger(),
                                SaveModelCallback(monitor='mae', comp=np.less, fname=f'best_fd002', reset_on_fit=True),
                                EarlyStoppingCallback(monitor='mae', patience=25)
                            ]
                        )
                        print(f"bs={bs} | layers={layers} | wd={wd} | drop={drop} | lr={lr}")
                        learn.fit_one_cycle(n_epochs, lr_max=lr)
                        val_metrics = learn.validate()
                        val_mae = float(val_metrics[1]) if len(val_metrics) > 1 else float('inf')
                        config_result = {
                            "bs": bs,
                            "layers": str(layers),
                            "epochs": n_epochs,
                            "lr": float(lr),
                            "wd": float(wd),
                            "drop": float(drop),
                            "val_mae": val_mae
                        }
                        all_results.append(config_result)
                        if val_mae < best_mae:
                            best_mae = val_mae
                            best_config = config_result.copy()
                        print(f"    -> Validation MAE: {val_mae:.2f}")
                    except Exception as e:
                        print(f"  SKIP config due to error: {e}")
                        continue

print(f"\nBest config for FD002: {best_config}, Best Validation MAE: {best_mae}")
print(f"Best model for FD002 saved as 'best_fd002.pth'")

# --- SAVE RESULTS TO CSV ---
results_df = pd.DataFrame(all_results).sort_values("val_mae")
results_df.to_csv(f"{base_dir}/fd002_grid_search_leaderboard.csv", index=False)
print("\n=== FD002 Grid Search Leaderboard (Top 10) ===")
print(results_df.head(10))



=== Full Grid/Random Search for FD002 (mirrors Optuna space) ===
  FD002 | bs=32 | valid set size: 10751
bs=32 | layers=[800, 400, 200] | wd=1e-05 | drop=0.08 | lr=0.0003


epoch,train_loss,valid_loss,mae,time
0,15790.001953,15461.434570,107.987549,00:24
1,14123.421875,14145.621094,106.214256,00:22
2,11749.999023,11170.509766,97.483429,00:23
3,7959.453125,7234.998047,75.801765,00:24
4,2583.428467,2351.168701,35.298462,00:22
5,1651.228394,1664.769653,31.184427,00:24
6,1732.074829,1448.094604,27.948845,00:24
7,1598.560181,1439.478149,28.269228,00:22
8,1534.506470,1435.871460,28.539965,00:23
9,1687.859619,1463.936768,28.998919,00:23


Better model found at epoch 0 with mae value: 107.987548828125.
Better model found at epoch 1 with mae value: 106.2142562866211.
Better model found at epoch 2 with mae value: 97.48342895507812.
Better model found at epoch 3 with mae value: 75.80176544189453.
Better model found at epoch 4 with mae value: 35.2984619140625.
Better model found at epoch 5 with mae value: 31.18442726135254.
Better model found at epoch 6 with mae value: 27.94884490966797.
Better model found at epoch 11 with mae value: 27.362783432006836.
Better model found at epoch 19 with mae value: 27.18497085571289.
Better model found at epoch 22 with mae value: 26.920135498046875.
Better model found at epoch 23 with mae value: 26.876171112060547.
No improvement since epoch 0: early stopping


  SKIP config due to error: Exception occured in `SaveModelCallback` when calling event `after_epoch`:
	list index out of range
bs=32 | layers=[800, 400, 200] | wd=1e-05 | drop=0.08 | lr=0.001


epoch,train_loss,valid_loss,mae,time
0,14217.140625,14118.016602,105.957329,00:23
1,10512.613281,10375.006836,94.087830,00:23
2,2967.212646,2166.164551,33.989136,00:22
3,1683.201050,1581.950317,28.901533,00:23
4,1575.750000,1532.161377,29.867434,00:24
5,1630.924683,1551.230713,29.014307,00:22
6,1566.784668,1446.147949,28.378738,00:23
7,1511.915039,1484.807373,28.586338,00:24
8,1542.325317,1419.225098,27.989489,00:22
9,1430.882690,1473.220337,27.670242,00:22


...

epoch,train_loss,valid_loss,mae,time
0,12169.362305,11762.051758,98.713387,00:20
1,3117.182373,2425.094482,35.372547,00:18
2,1681.796509,1558.207031,29.166883,00:19
3,1662.492554,1524.299072,29.094893,00:19
4,1669.231323,1632.331665,30.316843,00:18
5,1579.667603,1437.951294,27.805994,00:19
6,1682.922363,1666.975708,31.040077,00:18
7,1606.924561,1530.449951,28.579954,00:18
8,1689.721802,1720.970093,30.626259,00:19
9,1562.625732,1456.664673,27.508081,00:18


Better model found at epoch 0 with mae value: 98.71338653564453.
Better model found at epoch 1 with mae value: 35.3725471496582.
Better model found at epoch 2 with mae value: 29.16688346862793.
Better model found at epoch 3 with mae value: 29.094892501831055.
Better model found at epoch 5 with mae value: 27.805994033813477.
Better model found at epoch 9 with mae value: 27.508081436157227.
Better model found at epoch 11 with mae value: 27.16923713684082.
Better model found at epoch 17 with mae value: 27.134807586669922.
Better model found at epoch 18 with mae value: 26.976030349731445.
Better model found at epoch 19 with mae value: 26.890920639038086.
Better model found at epoch 20 with mae value: 26.811941146850586.
Better model found at epoch 22 with mae value: 26.740808486938477.
No improvement since epoch 0: early stopping


  SKIP config due to error: Exception occured in `SaveModelCallback` when calling event `after_epoch`:
	list index out of range
bs=32 | layers=[512, 256, 128] | wd=3e-05 | drop=0.12 | lr=0.0003


epoch,train_loss,valid_loss,mae,time
0,15425.850586,15624.776367,107.194550,00:18
1,14818.632812,14678.161133,105.928299,00:19


Better model found at epoch 0 with mae value: 107.19454956054688.
Better model found at epoch 1 with mae value: 105.92829895019531.


In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import numpy as np
mse = mean_squared_error(y_val, preds)
rmse = np.sqrt(mse)

# 1. Pick your FD (e.g., FD001)
df = train_dfs[2].copy()

# 2. Define features and target
feature_cols = [col for col in df.columns if col not in ['id', 'cycle', 'rul']]  # exclude id/cycle/rul
X = df[feature_cols]
y = df['rul']

# 3. Train/test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Fit different models
models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'Linear Regression': LinearRegression()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    mse = mean_squared_error(y_val, preds)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_val, preds)
    print(f'{name}: RMSE={rmse:.2f}, R²={r2:.2f}')

# 5. (Optional) Feature importances for tree models
rf = models['Random Forest']
importances = pd.Series(rf.feature_importances_, index=feature_cols)
print("\nFeature Importances (Random Forest):\n", importances.sort_values(ascending=False))


NameError: name 'y_val' is not defined

In [ ]:
# 1. Install extra libraries if needed
!pip install xgboost lightgbm catboost --quiet

# 2. Imports
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# 3. Prepare data
df = train_dfs[2].copy()  # Use FD001 as example, repeat for others
feature_cols = [col for col in df.columns if col not in ['id', 'cycle', 'rul']]
X = df[feature_cols]
y = df['rul']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Define all models in a dictionary
models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'Linear Regression': LinearRegression(),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42, verbosity=0),
    'LightGBM': LGBMRegressor(n_estimators=100, random_state=42),
    'CatBoost': CatBoostRegressor(iterations=100, random_state=42, verbose=0),
    'MLPRegressor': MLPRegressor(hidden_layer_sizes=(100,50), max_iter=300, random_state=42),
    'SVR': SVR(kernel='rbf', C=1.0, epsilon=0.2),
    'KNN': KNeighborsRegressor(n_neighbors=5)
}

# 5. Training & Evaluation loop
results = []
for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    mae = mean_absolute_error(y_val, preds)
    mse = mean_squared_error(y_val, preds)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_val, preds)
    print(f'{name}: MAE={mae:.2f}, RMSE={rmse:.2f}, R²={r2:.2f}')
    results.append({'Model': name, 'MAE': mae, 'RMSE': rmse, 'R2': r2})

# 6. Show summary table
results_df = pd.DataFrame(results).sort_values('MAE')
print("\nModel Comparison Results:")
print(results_df)


Training Random Forest...
Random Forest: MAE=32.87, RMSE=44.44, R²=0.58

Training Gradient Boosting...
Gradient Boosting: MAE=34.35, RMSE=45.20, R²=0.56

Training Linear Regression...
Linear Regression: MAE=36.06, RMSE=46.85, R²=0.53

Training XGBoost...
XGBoost: MAE=33.20, RMSE=44.85, R²=0.57

Training LightGBM...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005396 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2853
[LightGBM] [Info] Number of data points in the train set: 43007, number of used features: 12
[LightGBM] [Info] Start training from score 108.245262
LightGBM: MAE=32.67, RMSE=43.93, R²=0.59

Training CatBoost...
CatBoost: MAE=33.05, RMSE=44.45, R²=0.58

Training MLPRegressor...
MLPRegressor: MAE=45.95, RMSE=63.04, R²=0.15

Training SVR...
SVR: MAE=56.23, RMSE=68.46, R²=0.00

Training KNN...
KNN: MAE=38.64, RMSE=51.37, R²=0.44

Model Comparison Results:
               Model        MA

In [ ]:
LightGBM Hyperparameter Tuning

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
param_grid_lgb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'num_leaves': [15, 31, 63]
}
lgb = LGBMRegressor(random_state=42)
grid_lgb = GridSearchCV(lgb, param_grid_lgb, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_lgb.fit(X_train, y_train)
best_lgb = grid_lgb.best_estimator_
print("LightGBM Best Params:", grid_lgb.best_params_)
print("LightGBM Validation MAE:", mean_absolute_error(y_val, best_lgb.predict(X_val)))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008740 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2853
[LightGBM] [Info] Number of data points in the train set: 43007, number of used features: 12
[LightGBM] [Info] Start training from score 108.245262
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

Gradient Boosting Hyperparameter Tuning

In [ ]:
param_grid_gb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0]
}
gb = GradientBoostingRegressor(random_state=42)
grid_gb = GridSearchCV(gb, param_grid_gb, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_gb.fit(X_train, y_train)
best_gb = grid_gb.best_estimator_
print("Gradient Boosting Best Params:", grid_gb.best_params_)
print("Gradient Boosting Validation MAE:", mean_absolute_error(y_val, best_gb.predict(X_val)))

CatBoost Hyperparameter Tuning

In [ ]:
param_grid_cat = {
    'iterations': [100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [3, 5, 7]
}
cat = CatBoostRegressor(random_state=42, verbose=0)
grid_cat = GridSearchCV(cat, param_grid_cat, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
grid_cat.fit(X_train, y_train)
best_cat = grid_cat.best_estimator_
print("CatBoost Best Params:", grid_cat.best_params_)
print("CatBoost Validation MAE:", mean_absolute_error(y_val, best_cat.predict(X_val)))


In [ ]:
# 1. Install and imports
!pip install --upgrade pip
!pip install fastai pandas numpy scikit-learn optuna --quiet

from fastai.tabular.all import *
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
import optuna
from google.colab import drive; drive.mount('/content/drive')

# 2. File paths for all FD001–FD004
base_dir    = '/content/drive/MyDrive/rul_project'
fds         = [2]
train_files = [f'{base_dir}/P_Train{fd}.csv' for fd in fds]
test_files  = [f'{base_dir}/P_Test{fd}.csv'  for fd in fds]

# 3. Load and store dataframes for all FDs in dictionaries
train_dfs = {}
test_dfs  = {}
for i, fd in enumerate(fds):
    tdf = pd.read_csv(train_files[i])
    tdf.columns = tdf.columns.str.strip().str.lower()
    train_dfs[fd] = tdf

    tsdf = pd.read_csv(test_files[i])
    tsdf.columns = tsdf.columns.str.strip().str.lower()
    test_dfs[fd] = tsdf

    print(f"\n==== FD00{fd} ====")
    print("Train columns:", train_dfs[fd].columns.tolist())
    print(train_dfs[fd].head())
    print("Test columns: ", test_dfs[fd].columns.tolist())
    print(test_dfs[fd].head())

# 4. Utility: group‐aware dataloaders by “id”
def get_dls_group(df, valid_frac=0.2, bs=32):
    # assume columns: 'id','cycle',<sensors...>,'rul'
    cont_names = [c for c in df.columns if c not in ('id','cycle','rul')]
    cat_names  = []
    # split so that all cycles of one engine ("id") stay together
    gss = GroupShuffleSplit(n_splits=1, test_size=valid_frac, random_state=42)
    train_idx, valid_idx = next(gss.split(df, groups=df.id))
    splits = (list(train_idx), list(valid_idx))
    to     = TabularPandas(df,
                           procs=[FillMissing, Normalize],
                           cat_names=cat_names,
                           cont_names=cont_names,
                           y_names='rul',
                           splits=splits)
    return to.dataloaders(bs=bs)

# 5. Single‐run training with LR‐finder, early stopping, weight‐decay, dropout
def train_one(df, bs, layers, ps, wd, max_epochs=50):
    dls = get_dls_group(df, bs=bs)
    config = tabular_config(ps=ps)  # Only dropout in config!
    learn = tabular_learner(dls,
                            layers=layers,    # Specify layers only here
                            config=config,
                            wd=wd,
                            loss_func=MSELossFlat(),
                            metrics=mae)
    lr_min, lr_steep = learn.lr_find(suggest_funcs=(minimum, steep))
    cbs = [EarlyStoppingCallback(monitor='mae', comp=np.less, patience=5)]
    learn.fit_one_cycle(max_epochs, lr_max=lr_min, cbs=cbs)
    return learn



# 6. Easy runner on any FD dataset
def run_experiment(df, name,
                   bs=32, layers=[200,100,50], ps=[0.2,0.1,0.05],
                   wd=1e-3, max_epochs=50):
    print(f"=== Running {name} ===")
    learn = train_one(df, bs, layers, ps, wd, max_epochs=max_epochs)
    val_mae = learn.validate()[1]
    print(f"{name} final validation MAE: {val_mae:.4f}")
    return learn

# Example usage:
learn002 = run_experiment(train_dfs[2], "FD002")
learn004 = run_experiment(train_dfs[4], "FD004")

# 7. (Optional) Optuna‐based hyperparameter search
def objective(trial, df):
    bs     = trial.suggest_categorical('bs',     [16,32,64])
    wd     = trial.suggest_loguniform(   'wd',     1e-5,1e-2)
    drop   = trial.suggest_uniform(      'drop',   0.1,0.5)
    layers = trial.suggest_categorical('layers', [[200,100],[400,200],[200,100,50]])
    ps     = [drop]*len(layers)
    learn  = train_one(df, bs, layers, ps, wd, max_epochs=30)
    return learn.validate()[1]

def run_optuna(df, name, n_trials=20):
    print(f"=== Optuna sweep on {name} ({n_trials} trials) ===")
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda t: objective(t, df), n_trials=n_trials)
    print(f"{name} best params: {study.best_params}")
    print(f"{name} best MAE:    {study.best_value:.4f}")
    return study

# Example Optuna usage:
study002 = run_optuna(train_dfs[2], "FD002", n_trials=25)
study004 = run_optuna(train_dfs[4], "FD004", n_trials=25)
